In [1]:
# Load data and define sharpe function to evaluate the models: 
#need volatility data and the data from the risk_adjusted_returns file:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#Get volatility historical data:
url_volatility = 'https://raw.githubusercontent.com/inga-maria01/master_thesis/main/data/volatility_data.csv'
volatility_df = pd.read_csv(url_volatility)
#make the date a datetime object:
volatility_df['Date'] = pd.to_datetime(volatility_df['Date'])

# first we need the historical data: 
url_nasdaq_price = 'https://raw.githubusercontent.com/inga-maria01/master_thesis/main/data/QQQ.csv'
price_df = pd.read_csv(url_nasdaq_price)
price_df['Date'] = pd.to_datetime(price_df['Date'])
price_df.head()
#this dataset includes the prices and the volume!
# we now need to calculate the returns:
price_df['r_ndq'] = (price_df['Adj Close'] - price_df['Adj Close'].shift(1)) / price_df['Adj Close'].shift(1)
price_df.dropna(inplace=True)
price_df.head()

rf_df = pd.read_csv('https://raw.githubusercontent.com/inga-maria01/master_thesis/main/data/IEF.csv')
rf_df['r_rf'] = (rf_df['Adj Close'] - rf_df['Adj Close'].shift(1)) / rf_df['Adj Close'].shift(1)
rf_df.dropna(inplace=True)
rf_df['Date'] = pd.to_datetime(rf_df['Date'])

portfolio_df = pd.merge(price_df[['Date', 'r_ndq']], 
                     rf_df[['Date', 'r_rf']], 
                     on='Date', 
                     how='inner')

portfolio_df = pd.merge(portfolio_df,  
                     volatility_df[['Date', 'Price']],
                     on='Date', 
                     how='inner')

portfolio_df.rename(columns={'Price' : 'Volatility'}, inplace = True)
portfolio_df.head()

r_rf_df = pd.read_csv('https://raw.githubusercontent.com/inga-maria01/master_thesis/main/data/risk_free_rate.csv')

#divide by 100 because it was already in % -> we dont want that here!
#then divide by 252 since its on a yearly basis but everything else is on a daily basis
#(252 ~ number of us trading days a year)
r_rf_df['daily_rf'] = r_rf_df['rf_rate']/(100*250)
r_rf_df['Date'] = pd.to_datetime(r_rf_df['Date'])
#r_rf_df.set_index('Date', inplace=True)
r_rf_df.head()

portfolio_df = pd.merge(portfolio_df, 
                        r_rf_df[['Date', 'rf_rate', 'daily_rf']], 
                        on='Date', 
                        how='inner')

sentiment_index_df = pd.read_excel('https://raw.githubusercontent.com/inga-maria01/master_thesis/main/index/sentiment_index_unweighted_v7.xlsx')
sentiment_index_df.rename(columns={'date':'Date'}, inplace=True)
# sentiment_index_df['lag_sentiment'] = sentiment_index_df['sentiment_score'].shift(1)
sentiment_index_df.head()

portfolio_df_sent = pd.merge(portfolio_df, sentiment_index_df, on='Date', how='inner')
portfolio_df_sent.set_index('Date', inplace=True)
portfolio_df_sent['moving_average_10day'] = portfolio_df_sent['sentiment_score'].rolling(window = 10).mean() # maybe try exponential?
portfolio_df_sent['moving_average_5day'] = portfolio_df_sent['sentiment_score'].rolling(window = 5).mean() # maybe try exponential?
portfolio_df_sent['moving_average_20day'] = portfolio_df_sent['sentiment_score'].rolling(window = 20).mean() # maybe try exponential?
portfolio_df_sent['moving_average_30day'] = portfolio_df_sent['sentiment_score'].rolling(window = 30).mean() # maybe try exponential?
portfolio_df_sent['moving_average_15day'] = portfolio_df_sent['sentiment_score'].rolling(window = 15).mean() # maybe try exponential?

#want to just look at the training period for threshold optimization
#and then just at the test period for calculating the sharpe
portfolio_df_sent_train = portfolio_df_sent[:'2018-06-30'].copy()
portfolio_df_sent_test = portfolio_df_sent['2018-07-01':].copy()


data_df = pd.merge(portfolio_df_sent, price_df[['Date', 'Close', 'Adj Close']], on='Date', how='inner')
data_df = data_df.rename(columns={'Close': 'close_qqq', 'Adj Close': 'adj_close_qqq'})



In [2]:
# future MA includes today as today's sentiment is still unknown before the trading day 
portfolio_df_sent['sentiment_ma_next_10'] = portfolio_df_sent['sentiment_score'].rolling(window=10).mean().shift(-9)

portfolio_df_sent['sentiment_ma_next_5'] = portfolio_df_sent['sentiment_score'].rolling(window=5).mean().shift(-4)

portfolio_df_sent['sentiment_ma_next_15'] = portfolio_df_sent['sentiment_score'].rolling(window=15).mean().shift(-14)


In [3]:
# ma_past = portfolio_df_sent.copy()
# ma_past = ma_past.drop(columns='sentiment_ma_next_10')
# ma_past = ma_past.dropna()

ma_future = portfolio_df_sent.copy()
ma_future = ma_future[ma_future['sentiment_ma_next_10'].notna()]


In [4]:
# ma_past = portfolio_df_sent.copy()
# ma_past = ma_past.drop(columns='sentiment_ma_next_10')
# ma_past = ma_past.dropna()

ma_future_15day = portfolio_df_sent.copy()
ma_future_15day = ma_future_15day[ma_future_15day['sentiment_ma_next_15'].notna()]
ma_future_15day = ma_future_15day[ma_future_15day['moving_average_15day'].notna()]

In [5]:
# ma_past = portfolio_df_sent.copy()
# ma_past = ma_past.drop(columns='sentiment_ma_next_10')
# ma_past = ma_past.dropna()

ma_future_5day = portfolio_df_sent.copy()
ma_future_5day = ma_future_5day[ma_future_5day['sentiment_ma_next_5'].notna()]

In [14]:
def sharpe_correct_10day(sentiment, version): # version is either sentiment_score or moving_average_10day
    w_ndq = []
    w_rf = []

    # Assuming 'sentiment' is a pandas Series
    
    upper = ma_future[version].shift(1).rolling(95).quantile(0.75) #apply(lambda x: weighted_quantile(x, quantile=0.75), raw=True)
    lower = ma_future[version].shift(1).rolling(95).quantile(0.25) #apply(lambda x: weighted_quantile(x, quantile=0.25), raw=True)
    test_sent = sentiment['2018-07-01':]
    test_df = ma_future['2018-07-01':].copy()
    for i in test_sent.index:
        score = sentiment[i]
        if score > upper[i]: #23: changing to see if it improves
            w_ndq.append(0.8)
            w_rf.append(0.2)
        elif score <= upper[i] and score > lower[i]: # was 20:
            w_ndq.append(0.6)
            w_rf.append(0.4)
        elif score <= lower[i]:
            w_ndq.append(0.2)
            w_rf.append(0.8)
    test_df['w_ndq'] = w_ndq
    test_df['w_rf'] = w_rf
    test_df['r_portfolio'] = test_df['w_ndq'] * test_df['r_ndq'] + test_df['w_rf'] * test_df['r_rf']
    test_df['excess returns'] = test_df['r_portfolio'] - test_df['daily_rf']
    test_df['trading day'] = (test_df['w_ndq'] != test_df['w_ndq'].shift(1)).astype(int)
    
    test_df.reset_index(inplace=True)
    # Identify maximum index in the dataset
    max_index = test_df.index.max()

    # Initialize starting portfolio value
    initial_portfolio_value = 1

    # Create a new DataFrame to store calculated values
    df_calc = pd.DataFrame(index=test_df.index, columns=['portfolio_value', 'ndq_start', 'rf_start', 'ndq_end', 'rf_end'])
    
    # Initialize first values from row 0
    df_calc.loc[0, 'portfolio_value'] = initial_portfolio_value
    df_calc.loc[0, 'ndq_start'] = 0
    df_calc.loc[0, 'rf_start'] = 0
    df_calc.loc[0, 'ndq_end'] = w_ndq[0] * initial_portfolio_value
    df_calc.loc[0, 'rf_end'] = w_rf[0] * initial_portfolio_value

    # Loop through rows from 1 to max_index to calculate the required columns
    for i in range(1, max_index + 1):
        # Calculate ndq_start and rf_start
        df_calc.loc[i, 'ndq_start'] = df_calc.loc[i - 1, 'ndq_end'] * (1 + test_df.loc[i, 'r_ndq'])
        df_calc.loc[i, 'rf_start'] = df_calc.loc[i - 1, 'rf_end'] * (1 + test_df.loc[i, 'r_rf'])

        # Calculate the new portfolio value
        df_calc.loc[i, 'portfolio_value'] = df_calc.loc[i, 'ndq_start'] + df_calc.loc[i, 'rf_start']

        # Calculate ndq_end and rf_end based on updated portfolio value
        df_calc.loc[i, 'ndq_end'] =  w_ndq[i] * df_calc.loc[i, 'portfolio_value']
        df_calc.loc[i, 'rf_end'] = w_rf[i] * df_calc.loc[i, 'portfolio_value']
    
    return2019 = (df_calc['portfolio_value'][-1:]/df_calc['portfolio_value'][0]) **(1/(365/250)) - 1
    risk_free2019 = test_df['rf_rate'].mean()/100
    std2019 = np.std(test_df['excess returns'])*np.sqrt(250)

    test_df.set_index('Date', inplace=True)
    
    return (return2019 - risk_free2019)/std2019, df_calc, test_df, upper, lower



In [7]:
def sharpe_correct_5day(sentiment, version): # version is either sentiment_score or moving_average_10day
    w_ndq = []
    w_rf = []

    # Assuming 'sentiment' is a pandas Series
    
    upper = ma_future_5day[version].shift(1).rolling(63).quantile(0.75) #apply(lambda x: weighted_quantile(x, quantile=0.75), raw=True)
    lower = ma_future_5day[version].shift(1).rolling(63).quantile(0.25) #apply(lambda x: weighted_quantile(x, quantile=0.25), raw=True)
    test_sent = sentiment['2018-07-01':]
    test_df = ma_future_5day['2018-07-01':].copy()
    for i in test_sent.index:
        score = sentiment[i]
        if score > upper[i]: #23: changing to see if it improves
            w_ndq.append(0.8)
            w_rf.append(0.2)
        elif score <= upper[i] and score > lower[i]: # was 20:
            w_ndq.append(0.6)
            w_rf.append(0.4)
        elif score <= lower[i]:
            w_ndq.append(0.2)
            w_rf.append(0.8)
    test_df['w_ndq'] = w_ndq
    test_df['w_rf'] = w_rf
    test_df['r_portfolio'] = test_df['w_ndq'] * test_df['r_ndq'] + test_df['w_rf'] * test_df['r_rf']
    test_df['excess returns'] = test_df['r_portfolio'] - test_df['daily_rf']
    test_df['trading day'] = (test_df['w_ndq'] != test_df['w_ndq'].shift(1)).astype(int)
    
    test_df.reset_index(inplace=True)
    # Identify maximum index in the dataset
    max_index = test_df.index.max()

    # Initialize starting portfolio value
    initial_portfolio_value = 1

    # Create a new DataFrame to store calculated values
    df_calc = pd.DataFrame(index=test_df.index, columns=['portfolio_value', 'ndq_start', 'rf_start', 'ndq_end', 'rf_end'])
    
    # Initialize first values from row 0
    df_calc.loc[0, 'portfolio_value'] = initial_portfolio_value
    df_calc.loc[0, 'ndq_start'] = 0
    df_calc.loc[0, 'rf_start'] = 0
    df_calc.loc[0, 'ndq_end'] = w_ndq[0] * initial_portfolio_value
    df_calc.loc[0, 'rf_end'] = w_rf[0] * initial_portfolio_value

    # Loop through rows from 1 to max_index to calculate the required columns
    for i in range(1, max_index + 1):
        # Calculate ndq_start and rf_start
        df_calc.loc[i, 'ndq_start'] = df_calc.loc[i - 1, 'ndq_end'] * (1 + test_df.loc[i, 'r_ndq'])
        df_calc.loc[i, 'rf_start'] = df_calc.loc[i - 1, 'rf_end'] * (1 + test_df.loc[i, 'r_rf'])

        # Calculate the new portfolio value
        df_calc.loc[i, 'portfolio_value'] = df_calc.loc[i, 'ndq_start'] + df_calc.loc[i, 'rf_start']

        # Calculate ndq_end and rf_end based on updated portfolio value
        df_calc.loc[i, 'ndq_end'] =  w_ndq[i] * df_calc.loc[i, 'portfolio_value']
        df_calc.loc[i, 'rf_end'] = w_rf[i] * df_calc.loc[i, 'portfolio_value']
    
    return2019 = (df_calc['portfolio_value'][-1:]/df_calc['portfolio_value'][0]) **(1/(365/250)) - 1
    risk_free2019 = test_df['rf_rate'].mean()/100
    std2019 = np.std(test_df['excess returns'])*np.sqrt(250)

    test_df.set_index('Date', inplace=True)
    
    return (return2019 - risk_free2019)/std2019, df_calc, test_df, upper, lower

In [8]:
def sharpe_correct_15day(sentiment, version): # version is either sentiment_score or moving_average_10day
    w_ndq = []
    w_rf = []

    # Assuming 'sentiment' is a pandas Series
    
    upper = ma_future_15day[version].shift(1).rolling(63).quantile(0.75) #apply(lambda x: weighted_quantile(x, quantile=0.75), raw=True)
    lower = ma_future_15day[version].shift(1).rolling(63).quantile(0.25) #apply(lambda x: weighted_quantile(x, quantile=0.25), raw=True)
    test_sent = sentiment['2018-07-01':]
    test_df = ma_future_15day['2018-07-01':].copy()
    for i in test_sent.index:
        score = sentiment[i]
        if score > upper[i]: #23: changing to see if it improves
            w_ndq.append(0.8)
            w_rf.append(0.2)
        elif score <= upper[i] and score > lower[i]: # was 20:
            w_ndq.append(0.6)
            w_rf.append(0.4)
        elif score <= lower[i]:
            w_ndq.append(0.2)
            w_rf.append(0.8)
    test_df['w_ndq'] = w_ndq
    test_df['w_rf'] = w_rf
    test_df['r_portfolio'] = test_df['w_ndq'] * test_df['r_ndq'] + test_df['w_rf'] * test_df['r_rf']
    test_df['excess returns'] = test_df['r_portfolio'] - test_df['daily_rf']
    test_df['trading day'] = (test_df['w_ndq'] != test_df['w_ndq'].shift(1)).astype(int)
    
    test_df.reset_index(inplace=True)
    # Identify maximum index in the dataset
    max_index = test_df.index.max()

    # Initialize starting portfolio value
    initial_portfolio_value = 1

    # Create a new DataFrame to store calculated values
    df_calc = pd.DataFrame(index=test_df.index, columns=['portfolio_value', 'ndq_start', 'rf_start', 'ndq_end', 'rf_end'])
    
    # Initialize first values from row 0
    df_calc.loc[0, 'portfolio_value'] = initial_portfolio_value
    df_calc.loc[0, 'ndq_start'] = 0
    df_calc.loc[0, 'rf_start'] = 0
    df_calc.loc[0, 'ndq_end'] = w_ndq[0] * initial_portfolio_value
    df_calc.loc[0, 'rf_end'] = w_rf[0] * initial_portfolio_value

    # Loop through rows from 1 to max_index to calculate the required columns
    for i in range(1, max_index + 1):
        # Calculate ndq_start and rf_start
        df_calc.loc[i, 'ndq_start'] = df_calc.loc[i - 1, 'ndq_end'] * (1 + test_df.loc[i, 'r_ndq'])
        df_calc.loc[i, 'rf_start'] = df_calc.loc[i - 1, 'rf_end'] * (1 + test_df.loc[i, 'r_rf'])

        # Calculate the new portfolio value
        df_calc.loc[i, 'portfolio_value'] = df_calc.loc[i, 'ndq_start'] + df_calc.loc[i, 'rf_start']

        # Calculate ndq_end and rf_end based on updated portfolio value
        df_calc.loc[i, 'ndq_end'] =  w_ndq[i] * df_calc.loc[i, 'portfolio_value']
        df_calc.loc[i, 'rf_end'] = w_rf[i] * df_calc.loc[i, 'portfolio_value']
    
    return2019 = (df_calc['portfolio_value'][-1:]/df_calc['portfolio_value'][0]) **(1/(365/250)) - 1
    risk_free2019 = test_df['rf_rate'].mean()/100
    std2019 = np.std(test_df['excess returns'])*np.sqrt(250)

    test_df.set_index('Date', inplace=True)
    
    return (return2019 - risk_free2019)/std2019, df_calc, test_df, upper, lower

In [10]:
sharpe_act_future_ma, df_calc_act_future_ma, test_df_act_future_ma, upper_act_future_ma, lower_act_future_ma = sharpe_correct_10day(ma_future['sentiment_ma_next_10'], 'moving_average_20day')
print(sharpe_act_future_ma)
print(test_df_act_future_ma['trading day'].sum())
#63 days results

363    3.442417
Name: portfolio_value, dtype: object
32


In [12]:
sharpe_act_future_ma, df_calc_act_future_ma, test_df_act_future_ma, upper_act_future_ma, lower_act_future_ma = sharpe_correct_10day(ma_future['sentiment_ma_next_10'], 'moving_average_20day')
print(sharpe_act_future_ma)
print(test_df_act_future_ma['trading day'].sum())
#126 days results

363    2.7689
Name: portfolio_value, dtype: object
27


In [15]:
sharpe_act_future_ma, df_calc_act_future_ma, test_df_act_future_ma, upper_act_future_ma, lower_act_future_ma = sharpe_correct_10day(ma_future['sentiment_ma_next_10'], 'moving_average_20day')
print(sharpe_act_future_ma)
print(test_df_act_future_ma['trading day'].sum())
#95 days results

363    2.965093
Name: portfolio_value, dtype: object
30


In [18]:
sharpe_act_future_ma, df_calc_act_future_ma, test_df_act_future_ma, upper_act_future_ma, lower_act_future_ma = sharpe_correct_15day(ma_future_15day['sentiment_ma_next_15'], 'moving_average_30day')
print(sharpe_act_future_ma)
print(test_df_act_future_ma['trading day'].sum())
#95 days results

358    3.075126
Name: portfolio_value, dtype: object
24
